<a id='Table of Contents'></a><h1>Table of Contents</h1>

- <a href='#introduction'>Introduction</a> 
- <a href='#imports'>Import and install required libraries</a>  
- <a href='#set_fitting'>Setting of fitting</a>  
- <a href='#run_fit'>Running the fit</a>  
- <a href='#results'>Showing the results</a> 

(<a href='#Table of Contents'>Top</a>)<a id='introduction'></a><h2>Introduction</h2>

<div class="alert alert-block alert-info">
The objective of this notebook is to show how to use the isotropic rotational diffusion model `IsotropicRotationalDiffusion`.
</div>

The **reference data** were generated using the above model with the following parameters:  
- Radius = 1.10 Angstrom 
- D_rot = 0.125 meV

The model was convoluted with a Gaussian resolution function 
of FWHM = 0.1 meV, centered randomly in the range \[-0.01, +0.01\] meV.

Finally the data were sampled randomly from a Poisson distribution.

There is no background.

(<a href='#Table of Contents'>Top</a>)<a id='imports'></a><h2> Import python modules for plotting, fitting... </h2>

In [1]:
# Imported required libraries
from __future__ import print_function
import sys
import os

import ipywidgets

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# install bumps (if not already installed)
import pkgutil
if not pkgutil.find_loader("bumps"):
    !{sys.executable} -m pip install bumps

 (<a href='#Table of Contents'>Top</a>)<a id='set_fitting'></a><h2>Setting of fitting </h2>
<h3> load data, prepare data, create fitting model and set initial guesses </h3>

These settings are saved in a Python script, which will be used by Bumps to run the fit.

In [3]:
# Name of Python script containing the model
model_file = "model_fit_IsoRot.py"

In [4]:
%%writefile $model_file
import h5py
import QENSmodels
import numpy as np
from scipy.integrate import simps
from bumps.names import *

path_to_data = './data/'

# Read the sample
f = h5py.File(path_to_data + 'IsoRot_Sample.hdf', 'r')
hw = f['entry1']['data1']['X'][:]
q = f['entry1']['data1']['Y'][:]
sqw = np.transpose(f['entry1']['data1']['DATA'][:])
err = np.transpose(f['entry1']['data1']['errors'][:])
f.close()

# Read resolution
f = h5py.File(path_to_data + 'IsoRot_Resol.hdf', 'r')
res = np.transpose(f['entry1']['data1']['DATA'][:])
f.close()

# Force resolution function to have unit area
for i in range(len(q)):
    area = simps(res[:,i], hw)
    res[:,i] /= area   

# Fitting model 
def model_convol(x, q, scale=1, center=0, radius=1, DR=1, resolution=None):
    model = QENSmodels.sqwIsotropicRotationalDiffusion(x, q, scale, center, radius, DR)
    return np.convolve(model, resolution/resolution.sum(), mode='same')

# Fit
M = []

# First dataset: wavelength=5 Angstrom 
for i in range(len(q)):

    # Bumps fitting model
    Mq = Curve(model_convol, hw, sqw[:,i], err[:,i], q=q[i],          
               scale=1000, center=0.0, radius=1.0, DR=0.1, resolution=res[:, i])
    Mq.scale.range(0, 1e5)
    Mq.center.range(-0.1, 0.1)
    Mq.radius.range(0, 3)
    Mq.DR.range(0, 2)
    
    # Q-independent parameters
    if i == 0:
        QR = Mq.radius
        QDR = Mq.DR
    else:
        Mq.radius = QR
        Mq.DR = QDR
    M.append(Mq)
        
problem = FitProblem(M)

Writing model_fit_IsoRot.py


<h3> Choice of minimizer for bumps </h3>

In [5]:
w_choice_minimizer = ipywidgets.widgets.Dropdown(
    options={'Levenberg-Marquardt': "lm", 
             'Nelder-Mead Simplex': "amoeba", 
             'DREAM': "dream", 
             'Differential Evolution': "de", 
             'Quasi-Newton BFGS': "newton", 
             'Random Lines (experimental)': "rl", 
             'Particle Swarm (experimental)': "ps", 
             'Parallel Tempering (experimental)': "pt"},
    value="lm",
    description='Minimizer:',)

display(w_choice_minimizer)

RHJvcGRvd24oZGVzY3JpcHRpb249dSdNaW5pbWl6ZXI6JywgaW5kZXg9NCwgb3B0aW9ucz17J1F1YXNpLU5ld3RvbiBCRkdTJzogJ25ld3RvbicsICdSYW5kb20gTGluZXMgKGV4cGVyaW1lbnTigKY=


<h3> Setting for running bumps </h3>

In [6]:
# CHOICE OF MINIMIZER
minimiser = w_choice_minimizer.value

# NUMBER OF STEPS WHEN RUNNING THE FIT
steps = 100

# output folder to save thre results
output_folder = 'QENS'

(<a href='#Table of Contents'>Top</a>)<a id='run_fit'></a><h2>Running the fit</h2>

Run the fit using the *minimizer* defined above with a number of *steps* also specified above
The subfolder *output_folder* contains different results: plots and 
refined parameters

In [ ]:
%run -m bumps.cli $model_file --fit=$minimiser --steps=$steps --store=$output_folder

# /Users/celinedurniak/anaconda/envs/mynewenv/lib/python2.7/site-packages/bumps/cli.py model_fit_IsoRot.py --fit=lm --steps=100 --store=QENS
-- Model 0 
.DR = 0.1 in [0,2]
.center = 0 in [-0.1,0.1]
.q = 0.2
.radius = 1 in [0,3]
.scale = 1000 in [0,100000]

[chisq=1860.223(12), nllf=372975]
-- Model 1 
.DR = 0.1 in [0,2]
.center = 0 in [-0.1,0.1]
.q = 0.4
.radius = 1 in [0,3]
.scale = 1000 in [0,100000]

[chisq=1766.934(12), nllf=354270]
-- Model 2 
.DR = 0.1 in [0,2]
.center = 0 in [-0.1,0.1]
.q = 0.6
.radius = 1 in [0,3]
.scale = 1000 in [0,100000]

[chisq=1602.580(12), nllf=321317]
-- Model 3 
.DR = 0.1 in [0,2]
.center = 0 in [-0.1,0.1]
.q = 0.8
.radius = 1 in [0,3]
.scale = 1000 in [0,100000]

[chisq=1431.498(12), nllf=287015]
-- Model 4 
.DR = 0.1 in [0,2]
.center = 0 in [-0.1,0.1]
.q = 1
.radius = 1 in [0,3]
.scale = 1000 in [0,100000]

[chisq=1215.738(12), nllf=243756]
-- Model 5 
.DR = 0.1 in [0,2]
.center = 0 in [-0.1,0.1]
.q = 1.2
.radius = 1 in [0,3]
.scale = 1000 in [0,1000

(<a href='#Table of Contents'>Top</a>)<a id='results'></a><h2>Showing the results </h2>

In [ ]:
# DISPLAY SOME RESULTS (Bumps output)
par_file = os.path.splitext(model_file)[0] + '.par'
err_file = os.path.splitext(model_file)[0] + '.err'

with open(os.path.join('.', output_folder, par_file), 'r') as fid:
    for line in fid:
        parameter, value = line.split()
        print(parameter, value)
        
with open(os.path.join(os.getcwd(), output_folder, err_file), 'r') as ferr:
    for line in ferr:
        print(line.rsplit('\n')[0])

In [ ]:
# Plot the fitting results

from IPython.display import display, Image
relative_output_path = os.path.join('.', output_folder)
names = [f for f in os.listdir(relative_output_path) if f.endswith('.png')]

for name in names:
    display(Image(os.path.join(relative_output_path, name)))